In [ ]:
%%time
###################################################################################################
### IMPORTS AND CONSTANTS
###################################################################################################
from time import gmtime, strftime
from datetime import date, datetime
import sys
print(sys.version)
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import whyqd as qd
from whyqd.parsers import CoreParser, DataSourceParser

# PARSERS
core = CoreParser()
reader = DataSourceParser()

# SOURCE_DIRECTORY = path to downloaded source data
STANDARDS = ['genomics', 'proteomics', 'metabolomics', 'universal']
INDEX_FIELD = "FAIRsharing Record (DOI or URL)"
INDICATORS = {
    "genomics": "MOMSI_G:",
    "proteomics": "MOMSI_P:",
    "metabolomics": "MOMSI_M:",
    "universal": "MOMSI_U:",
}
DATASOURCE_PATH = "rda-momsi-standards.xlsx"
MIMETYPE = "xlsx"
SCHEMA_SOURCE = {}
DATA_SOURCE = {}

3.11.4 (main, Nov  7 2023, 16:51:51) [GCC 11.4.0]
2025-04-13 07:20:32


2025-04-13 09:20:44,651	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


CPU times: user 762 ms, sys: 1.92 s, total: 2.68 s
Wall time: 12 s


In [2]:
%%time
###################################################################################################
### RESTRUCTURE AS AN ARRAY OF OBJECTS
###################################################################################################
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

# Get data sources from multiple sheets and derive the source schemas for each
datasource = qd.DataSourceDefinition()
datasource.derive_model(source=DATASOURCE_PATH, mimetype=MIMETYPE)
for ds in datasource.get:
    # We separate out the individual sheets to transform independently
    key = ds.sheet_name.lower().split()[0]
    if key in STANDARDS:
        # Get the dataframe
        df = reader.get(source=ds)
        end = list(df.columns).index("FAIRsharing Record (DOI or URL)") + 1
        columns = df.columns[:end]
        # df = df[columns]
        df.dropna(subset=columns[:5], inplace=True)
        identifiers = [f"{INDICATORS[key]}{i:05}" for i in range(len(df))]
        df["Identifier"] = identifiers
        columns = columns.insert(0, "Identifier")
        # df["Contributor"] = None
        # columns = columns.insert(len(columns), "Contributor")
        df = df[columns]
        records = df.to_dict("records")
        for r in records:
            r["Contributor"] = None
            r["Contributor Name"] = None
            r["CRediT"] = None
            r["Contributor ORCID ID"] = None
        print(f"{key}: {list(columns)}")
        # Process for empty rows, and for the first 
        DATA_SOURCE[key] = records

# Show source schemas
print(DATA_SOURCE.keys())

core.save_json(data=DATA_SOURCE, source=str(SOURCE_DIRECTORY / MOMSI_DATABASE))

2025-04-13 07:20:48
Failed to delete /tmp/.X11-unix. Reason: [Errno 30] Read-only file system: 'X0'
Failed to delete /tmp/.XIM-unix. Reason: [Errno 1] Operation not permitted: '/tmp/.XIM-unix'
Failed to delete /tmp/.Test-unix. Reason: [Errno 1] Operation not permitted: '/tmp/.Test-unix'
Failed to delete /tmp/systemd-private-4c7c17e1d4f44cc880d7b7843cbdb540-systemd-resolved.service-NOThB1. Reason: [Errno 13] Permission denied: '/tmp/systemd-private-4c7c17e1d4f44cc880d7b7843cbdb540-systemd-resolved.service-NOThB1'
Failed to delete /tmp/systemd-private-4c7c17e1d4f44cc880d7b7843cbdb540-systemd-timesyncd.service-3g4fUk. Reason: [Errno 13] Permission denied: '/tmp/systemd-private-4c7c17e1d4f44cc880d7b7843cbdb540-systemd-timesyncd.service-3g4fUk'
Failed to delete /tmp/systemd-private-4c7c17e1d4f44cc880d7b7843cbdb540-systemd-logind.service-aDH4XS. Reason: [Errno 13] Permission denied: '/tmp/systemd-private-4c7c17e1d4f44cc880d7b7843cbdb540-systemd-logind.service-aDH4XS'
Failed to delete /tmp/.I

2025-04-13 09:20:55,939	INFO worker.py:1724 -- Started a local Ray instance.


genomics: ['Identifier', 'Standard Type', 'Status', 'Country', 'Domain Class/Subclass', 'Application Technology', 'Standard Name', 'Acronym/Short Name', 'Planning', 'Collection', 'Processing', 'Analysis', 'Preservation', 'Sharing', 'Reuse', 'Meets Criteria', 'Active Affiliation(s)', 'Homepage', 'Reference Article Citation (DOI)', 'Reference Source Code (DOI or URL)', 'FAIRsharing Record (DOI or URL)']
proteomics: ['Identifier', 'Standard Type', 'Status', 'Country', 'Domain Class/Subclass', 'Application Technology', 'Standard Name', 'Acronym/Short Name', 'Description', 'Planning', 'Collection', 'Processing', 'Analysis', 'Preservation', 'Sharing', 'Reuse', 'Meets Criteria', 'Active Affiliation(s)', 'Homepage', 'Reference Article Citation (DOI)', 'Reference Source Code (DOI or URL)', 'FAIRsharing Record (DOI or URL)']
metabolomics: ['Identifier', 'Standard Type', 'Status', 'Country', 'Domain Class/Subclass', 'Application Technology', 'Standard Name', 'Acronym/Short Name', 'Planning', 'Col

True